<a href="https://colab.research.google.com/github/kushaldahal0/document-query-chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unstructured[pdf]

In [10]:
!pip install langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.2 MB/s eta 0:00:00


In [3]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
from google.colab import userdata

In [21]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
from langchain.document_loaders import DirectoryLoader

In [12]:
# to upload document directly from user and keep in documents_path

from google.colab import files
import os

documents_path = 'user_documents/'
os.makedirs(documents_path, exist_ok=True)

uploaded = files.upload()

for filename in uploaded.keys():
  file_path = os.path.join(documents_path, filename)
  with open(file_path, 'wb') as f:
    f.write(uploaded[filename])
  print(f"Saved file {filename} to {file_path}")

Saving chapter 5.pdf to chapter 5.pdf
Saved file chapter 5.pdf to user_documents/chapter 5.pdf


In [2]:
import os
print(os.listdir('user_documents'))

['chapter 5.pdf']


In [8]:
def load_documents(directory_path, glob_pattern):
    try:
        loader = DirectoryLoader(path=directory_path, glob=glob_pattern)
        documents = loader.load()
        return documents
    except Exception as e:
        print(f"Error loading documents: {e}")
        return []

directory_path = 'user_documents/'
glob_pattern = '**/*.pdf'

documents = load_documents(directory_path, glob_pattern)

if not documents:
    print("No documents found.")
else:
    print(f"Loaded {len(documents)} documents.")



Loaded 1 documents.


In [10]:
!pip install langchain_google_genai

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
type(documents[0])

In [12]:
documents[0].page_content

'The methods of data collection and common\n\nsampling techniques. Bimal Pandey\n\nJune 2, 2024\n\n1\n\nContents\n\n1 Primary and Secondary data\n\n2 Methods of Collecting primary data\n\n2.1 Observation Method . . . . . . . . . . . . . . . . . . . . . . . Interview Method . . . . . . . . . . . . . . . . . . . . . . . . . 2.2 2.2.1 Personal Interviews . . . . . . . . . . . . . . . . . . . . 2.2.2 Telephone Interviews . . . . . . . . . . . . . . . . . . . 2.3 Collection of Data Through Questionnaires . . . . . . . . . . . . . . . . . . . . . . . . 2.4 Collection Of Data Through Schedules\n\n3 Preparation of Questionnaire\n\n4 Types of Questions\n\n10 4.1 Structured Questionnaire . . . . . . . . . . . . . . . . . . . . . 10 4.2 Unstructured Questionnaire . . . . . . . . . . . . . . . . . . . 11\n\n5 Characteristics Of Good Questionnaire\n\n11 5.1 Limited Number Of Questions . . . . . . . . . . . . . . . . . . 11 5.2 Proper Sequence Of Questions . . . . . . . . . . . . . . . . . . 11 5.3 

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_by_page(text):
  pages = []
  current_page = ""

  for line in text.splitlines():
    if "page" in line.lower():
      if current_page:
        pages.append(current_page)
      current_page = line + "\n"
    else:
      current_page += line + "\n"

  if current_page:
    pages.append(current_page)

  if len(pages) == 1:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 2)
    chunks = text_splitter.split_text(text)
    return chunks

  return pages

In [17]:
import numpy as np

# Replace 'your_api_key' with your actual API key
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GOOGLE_API_KEY)

# Assuming `document` is a single string containing the entire document content
pages = split_by_page(documents[0].page_content)
print("pages:",len(pages))

pages: 85


In [25]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.3 MB/s eta 0:00:00


In [23]:
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import GoogleGenerativeAIEmbeddings

def get_vector_store(chunks):
  embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
  vectorstore = FAISS.from_texts(texts=chunks, embedding = embeddings)
  vectorstore.save_local('faiss_index')

In [ ]:
# Create a conversational QA chain
from langchain.prompts import PromptTemplate
def get_conversational_chain():
  template = """ Answer the question as detailed as possible from the provided context, make sure to provide all details,
  if answer is not in context do not provide
  Context:
  \n{context}?\n
  Question: \n{question}\n
  Answer :
  """

  PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
  )

  qa_chain = load_qa_chain(llm=llm, chain_type="stuff",prompt = PROMPT)
  return qa_chain

In [35]:
# Define the handle_user_query function
def handle_user_query(user_query):
    if "call me" in user_query.lower():
        name = input("Please enter your name: ")
        phone = input("Please enter your phone number: ")
        email = input("Please enter your email: ")

        user_info = {
            'name': name,
            'phone': phone,
            'email': email
        }

        return f"Thank you {name}. We will contact you at {phone} or {email} shortly.", user_info

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
    new_db = FAISS.load_local('faiss_index', embeddings, allow_dangerous_deserialization = True)
    docs = new_db.similarity_search(user_query)
    chain = get_conversational_chain()


    response = chain(
        {"input_documents":docs, "question": user_query}
                     ,return_only_outputs =True)

    return response["output_text"], None

In [39]:
# Define the chatbot function
def chatbot():
    get_vector_store(pages)
    print("Welcome to the Document Query Chatbot!")
    while True:
        user_query = input("You: ")
        if "exit" in user_query.lower():
          return
        response, user_info = handle_user_query(user_query)
        print(f"Chatbot: {response}")

        if user_info:
            print("Collected User Information:", user_info)
            # Here you can handle storing or using the user information as needed

# Run the chatbot
chatbot()

Welcome to the Document Query Chatbot!
You: just give me basic summary of the entire pdf
Chatbot: This document appears to be a chapter from a textbook or research paper on statistics and sampling methods. It covers various aspects of research, including:

* **Fundamentals of Statistical Terms:**  Defines key terms like "universe," "population," "characteristics," and "elementary units." It also explains the concepts of finite and infinite populations.
* **Sampling Techniques:**  Explains two main categories of sampling:
    * **Probability Sampling:**  Describes methods like simple random sampling, systematic sampling, and stratified sampling. Provides an example of how to use these methods to select a sample of households for research.
    * **Non-Probability Sampling:**  Mentions this category but doesn't delve into specific methods. 
* **Sample Size Determination:**  Briefly mentions the importance of determining sample size for estimating the mean of a population.

The document se